In [153]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [154]:
df = pd.read_excel("../cleaned_dataset_no_percentile_outliers.xlsx")
df.drop(columns=['Carimbo de data/hora', 'sanitized_metrics'], inplace=True)

## Pre-processamento

In [155]:
# Juntar todas as colunas de texto que desejamos transformar em uma única coluna
textual_cols = ["role", "org_size", "org_field", "agile_methods", "use_metrics_planning", 
                "use_metrics_review", "use_metrics_weekly", "use_metrics_daily", 
                "use_metrics_retro", "use_metrics_na", "metrics_category"]

In [156]:
df["combined_text"] = df[textual_cols].fillna('').agg(' '.join, axis=1)

In [158]:
metric_cols = ["metric_1", "metric_2", "metric_3", "metric_4", "metric_5", "metric_6", 
               "metric_7", "metric_8", "metric_9", "metric_10", "metric_11", "metric_13", 
               "metric_14", "metric_15", "metric_16", "metric_17", "metric_18"]

In [159]:
all_metrics = pd.concat([df[metric_cols].stack()]).unique()

In [171]:
metrics_df = pd.DataFrame({'metric': all_metrics})
metrics_df.head()

,metric
0,Ideal team capacity
1,NPS
2,Burndown
3,Outstanding bugs
4,Throughput


In [165]:

all_text = pd.concat([df["combined_text"], metrics_df["metric"]], axis=0)

In [173]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
tfidf_vectorizer.fit(all_text)

TfidfVectorizer(max_features=500, stop_words='english')

In [175]:
# Passo 4: Transformar as características dos gerentes e as métricas em TF-IDF usando o mesmo vetorizador
X_text_tfidf = tfidf_vectorizer.transform(df["combined_text"])
metrics_tfidf = tfidf_vectorizer.transform(metrics_df["metric"])

## Treinamento

In [184]:
#  Calcular a similaridade do cosseno entre as características dos gerentes e as métricas
affinity_scores = cosine_similarity(X_text_tfidf, metrics_tfidf)

In [185]:
# Criar um DataFrame para armazenar os resultados de afinidade
affinity_df = pd.DataFrame(affinity_scores, index=df.index, columns=metrics_df['metric'])


In [186]:
# Passo 7: Exibir os resultados para ver a afinidade de cada gerente com cada métrica
affinity_df.head()

metric,Ideal team capacity,NPS,Burndown,Outstanding bugs,Throughput,Number of remaining tasks,OKR,Velocity,CAC,Roadmaps,...,CFD,Number of completed tasks,Churn,Quality rating,KPI,Team total available hours,Project avg. cost,CSAT,SLA,Dívidas técnicas
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.054609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.048351,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [187]:
# Passo 3.1: Características do novo gerente (supor que temos um novo gerente com as seguintes características)
new_manager_data = {
    'role': 'Project Manager',
    'org_size': 'Medium',
    'org_field': 'IT',
    'agile_methods': 'Scrum',
    'use_metrics_planning': 'Yes',
    'use_metrics_review': 'Yes',
    'use_metrics_weekly': 'No',
    'use_metrics_daily': 'Yes',
    'use_metrics_retro': 'No',
    'use_metrics_na': 'No',
    'metrics_category': 'Team'
}

# Passo 3.2: Combinar as características do novo gerente em uma única string
new_manager_combined_text = ' '.join([str(val) for val in new_manager_data.values()])

# Passo 3.3: Transformar as características do novo gerente em TF-IDF
new_manager_tfidf = tfidf_vectorizer.transform([new_manager_combined_text])

# Passo 3.4: Calcular a similaridade com as métricas
new_affinity_scores = cosine_similarity(new_manager_tfidf, metrics_tfidf)

# Passo 3.5: Criar um DataFrame para exibir as recomendações
new_affinity_df = pd.DataFrame(new_affinity_scores, columns=metrics_df['metric'])

# Exibir as afinidades para o novo gerente (ordenar da mais alta para a mais baixa)
recommendations = new_affinity_df.T.sort_values(by=0, ascending=False)
print("Recomendações de métricas para o novo gerente:")
print(recommendations)

Recomendações de métricas para o novo gerente:
                                             0
metric                                        
Project avg. cost                     0.288466
Ideal team capacity                   0.129127
Team and individual motivational      0.129127
Team total available hours            0.114331
Team total effective available hours  0.097931
NPS                                   0.000000
Burndown                              0.000000
Outstanding bugs                      0.000000
CAC                                   0.000000
Roadmaps                              0.000000
Conversão                             0.000000
User Story Points                     0.000000
Throughput                            0.000000
Number of remaining tasks             0.000000
OKR                                   0.000000
Velocity                              0.000000
Test Coverage                         0.000000
Security Test Pass Rate (security     0.000000
Lead time    